# Capstone Project  - The Battle of Neighborhoods

## Introduction

As we all know, Canada contains various regions and each region has different culture and environment. From my personal interest, I would like to take a closer look at the east and west coast of Canada, Nova Scotia and British Columbia. Both cities are very diverse and are the financial capitals of Canada. In this report, it would like to compare the neighborhoods of the two province and determine how similar or dissimilar they are. Based on the venues both cities have, people who have little understanding about these two cities would have a clearer picture of what those cities look like and what could be expected if they have chance to travel around.  

To solve the problem which mentioned above, K- Means will be a potential method to use.




## Data Extraction

In this report, it will use [Canada Cities Database](https://simplemaps.com/data/canada-cities) to extract city name and geolocation for Nova Scotiaand British Columbia then Foursquare will be applied to to do further exploration and analysis.


In the data set, it would contain city, geographical location and different kind of venues for Nova Scotia and British Columbia.



## Methodology

Methodology section which represents the main component of the report where you discuss and describe any **exploratory data analysis** that you did, any **inferential statistical testing** that you performed, if any, and what **machine learnings** were used and why.

### Data Input

Let's input the data from Canada Cities Database.

In [1]:
import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.

if os.environ.get('RUNTIME_ENV_LOCATION_TYPE') == 'external':
    endpoint_12eb873726bb473cb94a9fbfaf04e420 = 'https://s3-api.us-geo.objectstorage.softlayer.net'
else:
    endpoint_12eb873726bb473cb94a9fbfaf04e420 = 'https://s3-api.us-geo.objectstorage.service.networklayer.com'

client_12eb873726bb473cb94a9fbfaf04e420 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='nyWN55RmqIxdimngdz4ZAVKIltSps0COwZRBOLIK2yGO',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url=endpoint_12eb873726bb473cb94a9fbfaf04e420)

body = client_12eb873726bb473cb94a9fbfaf04e420.get_object(Bucket='applieddatasciencecapstone-donotdelete-pr-f6a2ubaax2c91w',Key='canadacities.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_raw = pd.read_csv(body)
df_raw.tail()


,city,city_ascii,province_id,province_name,lat,lng,population,density,timezone,ranking,postal,id
1733,Durham-Sud,Durham-Sud,QC,Quebec,45.6667,-72.3333,1008.0,10.8,America/Montreal,4,J0H,1124105436
1734,Melbourne,Melbourne,QC,Quebec,45.5800,-72.1700,1004.0,5.8,America/Montreal,3,J0B,1124850489
1735,Nipawin No. 487,Nipawin No. 487,SK,Saskatchewan,53.2881,-104.0544,1004.0,1.1,America/Regina,4,S0E,1124001339
1736,Duck Lake No. 463,Duck Lake No. 463,SK,Saskatchewan,52.9596,-106.2089,1004.0,1.0,America/Regina,4,S0K S6V,1124001661
1737,Oyen,Oyen,AB,Alberta,51.3522,-110.4739,1001.0,189.6,America/Edmonton,3,T0J,1124000494


In [2]:
#only select bc and ns data
bc_df = df_raw[df_raw["province_id"] == "BC"]
bc_df = bc_df[['city','province_id','lat','lng','population','density']]
ns_df = df_raw[df_raw["province_id"] == "NS"]
ns_df = ns_df[['city','province_id','lat','lng','population','density']]

print('The Nova Scotia has {} cities and British Columbia has {} cities.'.format(
        ns_df.shape[0],bc_df.shape[0]
    )
)

The Nova Scotia has 35 cities and British Columbia has 145 cities.


Combine those two dataframe together

In [3]:
df = ns_df.append(bc_df, ignore_index=True)

(180, 6)

In [5]:
!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library
import json
!conda install -c conda-forge geopy --yes
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

Solving environment: / 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - defaults/noarch::ibm-wsrt-py37main-main==custom=1962
  - conda-forge/linux-64::pytorch==1.8.0=cpu_py37hafa7651_0
  - defaults/noarch::ibm-wsrt-py37main-keep==0.0.0=1962
done

# All requested packages already installed.

Solving environment: | 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - defaults/noarch::ibm-wsrt-py37main-main==custom=1962
  - conda-forge/linux-64::pytorch==1.8.0=cpu_py37hafa7651_0
  - defaults/noarch::ibm-wsrt-py37main-keep==0.0.0=1962
done

# All requested packages already installed.



In [6]:
from geopy.geocoders import Nominatim
address = 'Canada'

geolocator = Nominatim(user_agent="ca_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Canada are {}, {}.'.format(latitude, longitude))
# create map of Canada using latitude and longitude values
map_ca = folium.Map(location=[latitude, longitude], zoom_start=3.5)

# add markers to map
for lat, lng, neighborhood,province in zip(df['lat'],df['lng'], df['city'],df['province_id']):
    label = '{},{}'.format(neighborhood,province)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_ca)  
    
map_ca

The geograpical coordinate of Canada are 61.0666922, -107.991707.


Next, we are going to start utilizing the Foursquare API to explore the cities and segment them.

In [7]:

CLIENT_ID = 'IXCLUK3R32MQVPJTRNO5X0I3J1QEK0OHB50KWR1XPZFQPL2R' # your Foursquare ID
CLIENT_SECRET = 'F0CKLHQC0RYF2KBZS1ZSJG1MD2GE1T2GVYJNMF3DZ0E5534U' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: IXCLUK3R32MQVPJTRNO5X0I3J1QEK0OHB50KWR1XPZFQPL2R
CLIENT_SECRET:F0CKLHQC0RYF2KBZS1ZSJG1MD2GE1T2GVYJNMF3DZ0E5534U


In [8]:
df.loc[0, 'city']
#df.loc[35, 'city']

'Halifax'

In [9]:
city_latitude = df.loc[35, 'lat'] # city latitude value
city_longitude = df.loc[35, 'lng'] # city longitude value

city_name = df.loc[35, 'city'] # city name

print('Latitude and longitude values of {} are {}, {}.'.format(city_name, 
                                                               city_latitude, 
                                                               city_longitude))

Latitude and longitude values of Vancouver are 49.25, -123.1.


In [10]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    city_latitude, 
    city_longitude, 
    radius, 
    LIMIT)

results = requests.get(url).json()

In [11]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [12]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  app.launch_new_instance()


,name,categories,lat,lng
0,The Acorn,Vegetarian / Vegan Restaurant,49.249442,-123.101105
1,Liberty Bakery + Cafe,Bakery,49.251882,-123.101113
2,The Fish Counter,Seafood Restaurant,49.250790,-123.101148
3,Hawker's Delight,Malay Restaurant,49.248383,-123.101230
4,Zakkushi Charcoal Grill,Japanese Restaurant,49.249117,-123.101378


In [14]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

61 venues were returned by Foursquare.


In [15]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

bc analysis

In [16]:
vancouver_venues = getNearbyVenues(names=bc_df['city'],
                                   latitudes=bc_df['lat'],
                                   longitudes=bc_df['lng']
                                  )

Vancouver
Surrey
Victoria
Burnaby
Richmond
Kelowna
Abbotsford
Coquitlam
Langley
Saanich
Delta
Kamloops
Nanaimo
North Vancouver
Chilliwack
White Rock
Maple Ridge
Prince George
New Westminster
Port Coquitlam
Vernon
Duncan
West Vancouver
Mission
Langford Station
Campbell River
Penticton
Port Moody
East Kelowna
North Cowichan
Courtenay
Fort St. John
Squamish
Cranbrook
Pitt Meadows
Oak Bay
Salmon Arm
Port Alberni
Esquimalt
Colwood
Central Saanich
Terrace
Comox
Powell River
Sooke
Dawson Creek
Lake Country
Parksville
Prince Rupert
Whistler
Sidney
Summerland
North Saanich
Williams Lake
Nelson
View Royal
Coldstream
Sechelt
Quesnel
Qualicum Beach
Ladysmith
Kitimat
Castlegar
Trail
Kimberley
Merritt
Hope
Kent
Peachland
Northern Rockies
Creston
Smithers
Fernie
Spallumcheen
Osoyoos
Oliver
Armstrong
Metchosin
Gibsons
Vanderhoof
Grand Forks
Port Hardy
Sparwood
Cumberland
Rossland
Mackenzie
Golden
Bowen Island
Fruitvale
Westbank
Lumby
Invermere
Lake Cowichan
Ellison
Houston
Enderby
Cedar
Princeton
Erri

In [17]:
print(vancouver_venues.shape)
vancouver_venues.head()

(1306, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Vancouver,49.25,-123.1,The Acorn,49.249442,-123.101105,Vegetarian / Vegan Restaurant
1,Vancouver,49.25,-123.1,Liberty Bakery + Cafe,49.251882,-123.101113,Bakery
2,Vancouver,49.25,-123.1,The Fish Counter,49.250790,-123.101148,Seafood Restaurant
3,Vancouver,49.25,-123.1,Hawker's Delight,49.248383,-123.101230,Malay Restaurant
4,Vancouver,49.25,-123.1,Zakkushi Charcoal Grill,49.249117,-123.101378,Japanese Restaurant


In [18]:
vancouver_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Abbotsford,28,28,28,28,28,28
Armstrong,4,4,4,4,4,4
Ashcroft,7,7,7,7,7,7
Barrière,1,1,1,1,1,1
Bowen Island,1,1,1,1,1,1
...,...,...,...,...,...,...
West Vancouver,1,1,1,1,1,1
Whistler,53,53,53,53,53,53
White Rock,27,27,27,27,27,27


In [19]:
print('There are {} uniques categories.'.format(len(vancouver_venues['Venue Category'].unique())))

There are 207 uniques categories.


In [20]:
halifax_venues = getNearbyVenues(names=ns_df['city'],
                                   latitudes=ns_df['lat'],
                                   longitudes=ns_df['lng']
                                  )


Halifax
Cape Breton
New Glasgow
Inverness
Truro
Kentville
Chester
Queens
Amherst
Bridgewater
Church Point
Argyle
Barrington
Yarmouth
Antigonish
Stellarton
Wolfville
Windsor
Westville
Port Hawkesbury
Pictou
Berwick
Trenton
Lunenburg
Lantz
Digby
Middleton
Shelburne
Stewiacke
Falmouth
Parrsboro
Oxford
Centreville
Wedgeport
Mahone Bay


In [21]:
print(halifax_venues.shape)
halifax_venues.head()

(162, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Halifax,44.6475,-63.5906,Dilly Daily (Dilly Dally Coffee Café),44.646484,-63.593344,Café
1,Halifax,44.6475,-63.5906,Sweet Hereafter,44.646183,-63.594538,Dessert Shop
2,Halifax,44.6475,-63.5906,Emera Oval,44.649300,-63.587236,Skating Rink
3,Halifax,44.6475,-63.5906,Robie St. Station,44.651019,-63.592565,Coffee Shop
4,Halifax,44.6475,-63.5906,Studio East food + drink,44.650698,-63.593012,Cambodian Restaurant


In [22]:
halifax_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Amherst,1,1,1,1,1,1
Antigonish,8,8,8,8,8,8
Berwick,4,4,4,4,4,4
Bridgewater,2,2,2,2,2,2
Cape Breton,22,22,22,22,22,22
Centreville,4,4,4,4,4,4
Church Point,2,2,2,2,2,2
Digby,13,13,13,13,13,13
Halifax,25,25,25,25,25,25


In [23]:
print('There are {} uniques categories.'.format(len(halifax_venues['Venue Category'].unique())))

There are 64 uniques categories.


Analyze Each Neighborhood

In [24]:
# one hot encoding
vancouver_onehot = pd.get_dummies(vancouver_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
vancouver_onehot['Neighborhood'] = vancouver_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [vancouver_onehot.columns[-1]] + list(vancouver_onehot.columns[:-1])
vancouver_onehot = vancouver_onehot[fixed_columns]

vancouver_onehot.head()

,Zoo,American Restaurant,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,Automotive Shop,BBQ Joint,...,Train Station,Travel & Transport,Tree,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Water Park,Wine Shop,Women's Store,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [25]:
# one hot encoding
halifax_onehot = pd.get_dummies(halifax_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
halifax_onehot['Neighborhood'] = halifax_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [halifax_onehot.columns[-1]] + list(halifax_onehot.columns[:-1])
halifax_onehot = halifax_onehot[fixed_columns]

halifax_onehot.head()

,Neighborhood,American Restaurant,Arts & Crafts Store,Bakery,Bank,Bed & Breakfast,Breakfast Spot,Brewery,Burger Joint,Café,...,Skating Rink,Soccer Field,Sushi Restaurant,Tea Room,Thai Restaurant,Trail,Train Station,Vegetarian / Vegan Restaurant,Waterfall,Women's Store
0,Halifax,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,Halifax,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Halifax,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
3,Halifax,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Halifax,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [26]:
vancouver_onehot.shape

(1306, 207)

In [27]:
halifax_onehot.shape

(162, 65)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category


In [28]:
vancouver_grouped = vancouver_onehot.groupby('Neighborhood').mean().reset_index()
vancouver_grouped.head()

,Neighborhood,Zoo,American Restaurant,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,Automotive Shop,...,Train Station,Travel & Transport,Tree,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Water Park,Wine Shop,Women's Store,Yoga Studio
0,Abbotsford,0.0,0.0,0.0,0.0,0.035714,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Armstrong,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Ashcroft,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Barrière,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Bowen Island,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [29]:
halifax_grouped = halifax_onehot.groupby('Neighborhood').mean().reset_index()
halifax_grouped.head()

,Neighborhood,American Restaurant,Arts & Crafts Store,Bakery,Bank,Bed & Breakfast,Breakfast Spot,Brewery,Burger Joint,Café,...,Skating Rink,Soccer Field,Sushi Restaurant,Tea Room,Thai Restaurant,Trail,Train Station,Vegetarian / Vegan Restaurant,Waterfall,Women's Store
0,Amherst,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000
1,Antigonish,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.125
2,Berwick,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.250000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000
3,Bridgewater,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000
4,Cape Breton,0.0,0.0,0.0,0.045455,0.0,0.0,0.0,0.0,0.090909,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000


In [31]:
vancouver_grouped.shape

(123, 207)

In [30]:
halifax_grouped.shape

(26, 65)

#### Let's print each neighborhood along with the top 5 most common venues


In [34]:
import numpy as np
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [36]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
vancouver_venues_sorted = pd.DataFrame(columns=columns)
vancouver_venues_sorted['Neighborhood'] = vancouver_grouped['Neighborhood']

for ind in np.arange(vancouver_grouped.shape[0]):
    vancouver_venues_sorted.iloc[ind, 1:] = return_most_common_venues(vancouver_grouped.iloc[ind, :], num_top_venues)

vancouver_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Abbotsford,Coffee Shop,Fast Food Restaurant,Grocery Store,Pharmacy,Sushi Restaurant,Breakfast Spot,Gym,Supermarket,Sporting Goods Shop,Cosmetics Shop
1,Armstrong,Café,Stadium,Breakfast Spot,Theme Park,Moroccan Restaurant,Motel,Motorsports Shop,Movie Theater,Multiplex,Museum
2,Ashcroft,Convenience Store,Bakery,Fast Food Restaurant,Restaurant,Construction & Landscaping,Zoo,Noodle House,Multiplex,Museum,Music School
3,Barrière,Coffee Shop,Pool,Motorsports Shop,Movie Theater,Multiplex,Museum,Music School,Music Store,Music Venue,National Park
4,Bowen Island,Scenic Lookout,Zoo,Plaza,Motel,Motorsports Shop,Movie Theater,Multiplex,Museum,Music School,Music Store


In [37]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
halifax_venues_sorted = pd.DataFrame(columns=columns)
halifax_venues_sorted['Neighborhood'] = halifax_grouped['Neighborhood']

for ind in np.arange(halifax_grouped.shape[0]):
    halifax_venues_sorted.iloc[ind, 1:] = return_most_common_venues(halifax_grouped.iloc[ind, :], num_top_venues)

halifax_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Amherst,Construction & Landscaping,American Restaurant,Port,Music Store,New American Restaurant,Other Great Outdoors,Outdoors & Recreation,Palace,Paper / Office Supplies Store,Park
1,Antigonish,Women's Store,Shopping Mall,Ice Cream Shop,Hotel,Grocery Store,Pet Store,Pizza Place,Liquor Store,Sushi Restaurant,Soccer Field
2,Berwick,Café,Grocery Store,Pizza Place,Pharmacy,Playground,Pet Store,Performing Arts Venue,Park,American Restaurant,Port
3,Bridgewater,Grocery Store,Pizza Place,American Restaurant,Liquor Store,Music Store,New American Restaurant,Other Great Outdoors,Outdoors & Recreation,Palace,Paper / Office Supplies Store
4,Cape Breton,Restaurant,Pizza Place,Café,Pub,Coffee Shop,Sandwich Place,Deli / Bodega,Hotel,Hobby Shop,Gym / Fitness Center


### 4. Cluster Neighborhoods


In [75]:
grouped = vancouver_grouped.append(halifax_grouped, ignore_index=True)
grouped = grouped.fillna(0)
grouped.head()

,Neighborhood,Zoo,American Restaurant,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,Automotive Shop,...,Bed & Breakfast,Cambodian Restaurant,Campground,Dumpling Restaurant,Hobby Shop,Outdoors & Recreation,Palace,Port,Soccer Field,Waterfall
0,Abbotsford,0.0,0.0,0.0,0.0,0.035714,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Armstrong,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Ashcroft,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Barrière,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Bowen Island,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [49]:
from sklearn.cluster import KMeans
# set number of clusters
kclusters = 5

grouped_clustering = grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 2, 4, 0, 0, 0, 0, 1,
       0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 4, 0, 3,
       0, 1, 4, 0, 0, 0, 0, 1, 1, 0, 0, 3, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1,
       1, 4, 0, 0, 0, 3, 0, 0, 0, 1, 1, 0, 1, 4, 1, 0, 0, 0, 0, 0, 1, 1,
       1, 0, 0, 1, 0, 0, 0, 3, 0, 0, 0, 0, 0, 4, 1, 1, 1, 0, 0, 0, 0, 0,
       0, 4, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 2, 0], dtype=int32)

In [66]:
# add clustering labels
venues_sorted = vancouver_venues_sorted.append(halifax_venues_sorted, ignore_index=True)
venues_sorted.insert(0, 'Cluster Labels',kmeans.labels_)
venues_sorted['Cluster Labels'] = venues_sorted['Cluster Labels'].apply(np.int64)
merged = df

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
merged = merged.join(venues_sorted.set_index('Neighborhood'), on='city')
merged.dropna(subset = ["Cluster Labels"], inplace=True)
merged['Cluster Labels'] = merged['Cluster Labels'].apply(np.int64)
merged.head() # check the last columns!


,city,province_id,lat,lng,population,density,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Halifax,NS,44.6475,-63.5906,403131.0,73.4,0,Coffee Shop,American Restaurant,Fast Food Restaurant,Liquor Store,Music Store,Other Great Outdoors,Outdoors & Recreation,Park,Performing Arts Venue,Pharmacy
1,Cape Breton,NS,46.1389,-60.1931,94285.0,38.8,0,Restaurant,Pizza Place,Café,Pub,Coffee Shop,Sandwich Place,Deli / Bodega,Hotel,Hobby Shop,Gym / Fitness Center
2,New Glasgow,NS,45.5926,-62.6455,20609.0,911.6,1,Italian Restaurant,Grocery Store,Bank,Convenience Store,Gastropub,Playground,Pizza Place,Pharmacy,Pet Store,Performing Arts Venue
4,Truro,NS,45.3647,-63.2800,12826.0,355.5,0,Bank,Pub,Fish & Chips Shop,Pizza Place,Train Station,Gastropub,Thai Restaurant,Pharmacy,Coffee Shop,Performing Arts Venue
5,Kentville,NS,45.0775,-64.4958,12088.0,363.3,0,Pub,Plaza,Bank,Brewery,Pharmacy,Park,Playground,Pizza Place,Pet Store,Performing Arts Venue


## Results

Here is the K Mean (K = 5)results shown in the map.

In [52]:
import matplotlib.cm as cm
import matplotlib.colors as colors

In [68]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(merged['lat'], merged['lng'], merged['city'], merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examine Clusters


Here is the detail cluster information. We could take a closer look at each cluster's result

In [69]:
merged.loc[merged['Cluster Labels'] == 0, merged.columns[[1] + list(range(5, merged.shape[1]))]]

,province_id,density,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,NS,73.4,0,Coffee Shop,American Restaurant,Fast Food Restaurant,Liquor Store,Music Store,Other Great Outdoors,Outdoors & Recreation,Park,Performing Arts Venue,Pharmacy
1,NS,38.8,0,Restaurant,Pizza Place,Café,Pub,Coffee Shop,Sandwich Place,Deli / Bodega,Hotel,Hobby Shop,Gym / Fitness Center
4,NS,355.5,0,Bank,Pub,Fish & Chips Shop,Pizza Place,Train Station,Gastropub,Thai Restaurant,Pharmacy,Coffee Shop,Performing Arts Venue
5,NS,363.3,0,Pub,Plaza,Bank,Brewery,Pharmacy,Park,Playground,Pizza Place,Pet Store,Performing Arts Venue
10,NS,9.4,0,Paper / Office Supplies Store,Food,American Restaurant,Liquor Store,Music Store,New American Restaurant,Other Great Outdoors,Outdoors & Recreation,Palace,Park
...,...,...,...,...,...,...,...,...,...,...,...,...,...
171,BC,191.9,0,Ice Cream Shop,Construction & Landscaping,Zoo,Nightlife Spot,Motorsports Shop,Movie Theater,Multiplex,Museum,Music School,Music Store
174,BC,466.2,0,Insurance Office,Construction & Landscaping,Café,Brewery,Sandwich Place,Electronics Store,Pet Store,Performing Arts Venue,Playground,Movie Theater
175,BC,104.0,0,Home Service,Zoo,Nightclub,Motel,Motorsports Shop,Movie Theater,Multiplex,Museum,Music School,Music Store
178,BC,197.6,0,Brewery,Café,Convenience Store,Asian Restaurant,Hockey Rink,Korean Restaurant,Music Venue,New American Restaurant,National Park,Zoo


In [70]:
merged.loc[merged['Cluster Labels'] == 1, merged.columns[[1] + list(range(5, merged.shape[1]))]]

,province_id,density,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,NS,911.6,1,Italian Restaurant,Grocery Store,Bank,Convenience Store,Gastropub,Playground,Pizza Place,Pharmacy,Pet Store,Performing Arts Venue
7,NS,4.3,1,Grocery Store,Construction & Landscaping,Campground,Performing Arts Venue,Playground,Pizza Place,Pharmacy,Pet Store,American Restaurant,Port
9,NS,625.9,1,Grocery Store,Pizza Place,American Restaurant,Liquor Store,Music Store,New American Restaurant,Other Great Outdoors,Outdoors & Recreation,Palace,Paper / Office Supplies Store
14,NS,871.7,1,Women's Store,Shopping Mall,Ice Cream Shop,Hotel,Grocery Store,Pet Store,Pizza Place,Liquor Store,Sushi Restaurant,Soccer Field
15,NS,468.1,1,Pizza Place,Breakfast Spot,Harbor / Marina,Grocery Store,American Restaurant,Park,Playground,Pharmacy,Pet Store,Performing Arts Venue
21,NS,381.3,1,Café,Grocery Store,Pizza Place,Pharmacy,Playground,Pet Store,Performing Arts Venue,Park,American Restaurant,Port
26,NS,329.1,1,Restaurant,Convenience Store,Sandwich Place,Pharmacy,American Restaurant,Performing Arts Venue,Pizza Place,Pet Store,Park,Plaza
28,NS,77.9,1,Grocery Store,Park,Restaurant,American Restaurant,Plaza,Music Store,New American Restaurant,Other Great Outdoors,Outdoors & Recreation,Palace
41,BC,376.5,1,Coffee Shop,Fast Food Restaurant,Grocery Store,Pharmacy,Sushi Restaurant,Breakfast Spot,Gym,Supermarket,Sporting Goods Shop,Cosmetics Shop
66,BC,820.2,1,Gas Station,Japanese Restaurant,Bank,Pizza Place,Pharmacy,Restaurant,Sandwich Place,Coffee Shop,Clothing Store,Café


In [71]:
merged.loc[merged['Cluster Labels'] == 2, merged.columns[[1] + list(range(5, merged.shape[1]))]]

,province_id,density,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
16,NS,649.8,2,Convenience Store,American Restaurant,Port,Music Store,New American Restaurant,Other Great Outdoors,Outdoors & Recreation,Palace,Paper / Office Supplies Store,Park
74,BC,954.2,2,Convenience Store,Zoo,Nightlife Spot,Motorsports Shop,Movie Theater,Multiplex,Museum,Music School,Music Store,Music Venue


In [72]:
merged.loc[merged['Cluster Labels'] == 3, merged.columns[[1] + list(range(5, merged.shape[1]))]]

,province_id,density,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
129,BC,41.0,3,Sandwich Place,Plaza,Motel,Motorsports Shop,Movie Theater,Multiplex,Museum,Music School,Music Store,Music Venue
139,BC,84.4,3,Sandwich Place,Motel,Pub,Nightclub,Motorsports Shop,Movie Theater,Multiplex,Museum,Music School,Music Store
152,BC,910.0,3,Motel,Pub,Zoo,Nightclub,Motorsports Shop,Movie Theater,Multiplex,Museum,Music School,Music Store
156,BC,361.3,3,Pub,Zoo,Nightclub,Motel,Motorsports Shop,Movie Theater,Multiplex,Museum,Music School,Music Store


In [74]:
merged.loc[merged['Cluster Labels'] == 4, merged.columns[[1] + list(range(5, merged.shape[1]))]]

,province_id,density,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,NS,779.7,4,Construction & Landscaping,American Restaurant,Port,Music Store,New American Restaurant,Other Great Outdoors,Outdoors & Recreation,Palace,Paper / Office Supplies Store,Park
24,NS,749.9,4,Cosmetics Shop,Construction & Landscaping,American Restaurant,Performing Arts Venue,Playground,Pizza Place,Pharmacy,Pet Store,Park,Port
77,BC,838.2,4,Construction & Landscaping,Zoo,Nightlife Spot,Motorsports Shop,Movie Theater,Multiplex,Museum,Music School,Music Store,Music Venue
79,BC,229.6,4,Construction & Landscaping,Zoo,Nightlife Spot,Motorsports Shop,Movie Theater,Multiplex,Museum,Music School,Music Store,Music Venue
94,BC,497.4,4,Construction & Landscaping,Restaurant,Zoo,Nightlife Spot,Motorsports Shop,Movie Theater,Multiplex,Museum,Music School,Music Store
162,BC,53.0,4,Furniture / Home Store,Construction & Landscaping,Zoo,Nightclub,Motorsports Shop,Movie Theater,Multiplex,Museum,Music School,Music Store
173,BC,278.7,4,Construction & Landscaping,Zoo,Nightlife Spot,Motorsports Shop,Movie Theater,Multiplex,Museum,Music School,Music Store,Music Venue


## Discussion

Frome the Result session, Nova Scotia and British Columnbia are mixed together in each cluster except for cluster 3 which only contains BC's cities. 

Cluster 0 and 1 is more about restaurants and groceries 

Cluster 2 is about convenience store; Cluster 3 is focus on eating places and also Music and Museum

Cluster 4 is more focused on Construction related venues

## Conclusion

From all the analysis it is applied, using K-Means method is hard to tell the difference bewteen Nova Scotia and British Columnbia for the reason that all 5 clusters all contain those two provinces' cities. The conlusion 

**Future improvement**: Try different number of cluster to see whether the results would turn better; Add more potential variables like population, population density and crime rate in each city.